In [2]:
import os
import json
import pandas as pd
import traceback


In [3]:
from langchain.chat_models import ChatOpenAI

In [4]:
from dotenv import load_dotenv
load_dotenv()
OPENAI_KEY = os.getenv('key')

In [5]:
llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model_name="gpt-3.5-turbo",temperature=0.3)

/Users/king610160/Desktop/python/mcqgen/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## Empty markdown cell, double click or press enter to edit

In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [8]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [9]:
quiz_generate_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generate_prompt,output_key='quiz',verbose=True)

In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE
)

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], 
                                        input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], 
                                        verbose=True,)

In [15]:
target_directory = '/Users/king610160/Desktop/python/mcqgen/'
os.chdir(target_directory)

file_path='data.txt'
with open(file_path, 'r') as file:
    TEXT = file.read()

In [16]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [17]:
NUMBER=5
SUBJECT='maching learning'
TONE='simple'

In [18]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
            {
                "text": TEXT,
                "number": NUMBER,
                "subject":SUBJECT,
                "tone": TONE,
                "response_json": json.dumps(RESPONSE_JSON)
            }
        )

/Users/king610160/Desktop/python/mcqgen/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [19]:
print(cb)

Tokens Used: 1882
	Prompt Tokens: 1194
	Completion Tokens: 688
Successful Requests: 2
Total Cost (USD): $0.003167


In [36]:
quiz = response.get('quiz')

In [37]:
# replace some digits
quiz = quiz.replace('\n', '')

In [38]:
quiz = json.loads(quiz)

In [42]:
quiz

{'1': {'mcq': 'What is biology?',
  'options': {'a': 'The study of rocks',
   'b': 'The scientific study of life',
   'c': 'The study of outer space',
   'd': 'The study of economics'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Chemistry',
   'c': 'Physics',
   'd': 'Genetics'},
  'correct': 'a'},
 '3': {'mcq': 'What is the importance of energy processing in organisms?',
  'options': {'a': 'It allows organisms to communicate',
   'b': 'It allows organisms to move, grow, and reproduce',
   'c': 'It allows organisms to hibernate',
   'd': 'It allows organisms to fly'},
  'correct': 'b'},
 '4': {'mcq': 'What is a major characteristic of all organisms?',
  'options': {'a': 'They can live forever',
   'b': 'They are made up of cells',
   'c': 'They can survive without water',
   'd': 'They do not reproduce'},
  'correct': 'b'},
 '5': {'mcq': 'What is the study of life at multiple levels of or

In [40]:
type(quiz)

dict

In [41]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [25]:
quiz_table_data

[{'MCQ': 'Whatisbiology?',
  'Choices': 'a: Thestudyofrocks | b: Thescientificstudyoflife | c: Thestudyofouterspace | d: Thestudyofeconomics',
  'Correct': 'b'},
 {'MCQ': 'Whichthemeinbiologyexplainstheunityanddiversityoflife?',
  'Choices': 'a: Evolution | b: Chemistry | c: Physics | d: Genetics',
  'Correct': 'a'},
 {'MCQ': 'Whatistheimportanceofenergyprocessinginorganisms?',
  'Choices': 'a: Itallowsorganismstocommunicate | b: Itallowsorganismstomove,grow,andreproduce | c: Itallowsorganismstohibernate | d: Itallowsorganismstofly',
  'Correct': 'b'},
 {'MCQ': 'Whatisamajorcharacteristicofallorganisms?',
  'Choices': 'a: Theycanliveforever | b: Theyaremadeupofcells | c: Theycansurvivewithoutwater | d: Theydonotreproduce',
  'Correct': 'b'},
 {'MCQ': 'Whatisthestudyoflifeatmultiplelevelsoforganizationcalled?',
  'Choices': 'a: Macrobiology | b: Microbiology | c: Molecularbiology | d: Astrobiology',
  'Correct': 'c'}]

In [26]:
quiz=pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("biology.csv",index=False)